- 简单demo示例 银行后台存取功能

In [1]:
def deposit():
    print('存款中')
def withdrawal():
    print('取款中')
    
button = 1
if button == 1:
    deposit()
else:
    withdrawal()


存款中


In [2]:
def deposit():
    print('存款中')
def withdrawal():
    print('取款中')
    
button = 2
if button == 1:
    deposit()
else:
    withdrawal()


取款中


- 添加密码验证功能

In [ ]:
def deposit():
    print('存款中')
def withdrawal():
    print('取款中')
def check_password():
    print('密码验证中....')

    
button = 2
if button == 1:
    check_password()
    deposit()
else:
    check_password()
    withdrawal()

- 添加密码功能（2）让应用代码更简洁

In [ ]:
def deposit():
    check_password()
    print('存款中')
def withdrawal():
    check_password()
    print('取款中')
def check_password():
    print('密码验证中....')
    
button = 2
if button == 1:
    deposit()
else:
    withdrawal()

- 使用回调函数，简化底层代码

In [ ]:
def check_password(func):
    print("密码验证中...")
    func()
    
def deposit():
    print("存款中...")
    
def withdrawal():
    print("取款中...")

button = 2
if button == 1:
    check_password(deposit)
    #deposit()
else:
    check_password(withdrawal)

- 不想动业务代码，为了直接调用，于是出现了装饰器雏形...

In [1]:
def check_password(func):
    print("密码验证中...")
    func()
    
def deposit():
    print("存款中...")
    
def withdrawal():
    print("取款中...")
    
deposit = check_password(deposit)    
withdrawal = check_password(withdrawal)   

button = 1
if button == 1:
    deposit()
else: 
    withdrawal()

密码验证中...
存款中...
密码验证中...
取款中...


TypeError: 'NoneType' object is not callable

In [2]:
# 易懂的装饰器简例
def check_password(func):
    def inner():
        print("密码验证中...")
        func()
    return inner
    
def deposit():
    print("存款中...")
    
def withdrawal():
    print("取款中...")
    
deposit = check_password(deposit)    
withdrawal = check_password(withdrawal)    

button = 2
if button == 1:
    deposit()
else:
    withdrawal()

密码验证中...
取款中...


In [4]:
#语法糖
def check_password(func):
    def inner():
        print("密码验证中...")
        func()
    return inner

@check_password      #deposit = check_password(deposit)
def deposit():
    print("存款中...")
@check_password      #withdrawal = check_password(withdrawal)  
def withdrawal():
    print("取款中...")
      
button = 2
if button == 1:
    deposit()
else:
    withdrawal()

密码验证中...
取款中...


---------- summary python装饰器本质上就是一个函数，它可以让其他函数在不需要做任何代码变动的前提下增加额外的功能，装饰器的返回值也是一个函数对象（函数的指针）。装饰器函数的外部函数传入我要装饰的函数名字，返回经过修饰后函数的名字；内层函数（闭包）负责修饰被修饰函数。从上面这段描述中我们需要记住装饰器的几点属性，以便后面能更好的理解：

　　　　实质： 是一个函数

　　　　参数：是你要装饰的函数名（并非函数调用）

　　　　返回：是装饰完的函数名（也非函数调用）

　　　　作用：为已经存在的对象添加额外的功能

　　　　特点：不需要对对象做任何的代码上的变动

python装饰器有很多经典的应用场景，比如：插入日志、性能测试、事务处理、权限校验等。装饰器是解决这类问题的绝佳设计。并且从引入中的列子中我们也可以归纳出：装饰器最大的作用就是对于我们已经写好的程序，我们可以抽离出一些雷同的代码组建多个特定功能的装饰器，这样我们就可以针对不同的需求去使用特定的装饰器，这时因为源码去除了大量泛化的内容而使得源码具有更加清晰的逻辑。

## 练习：统计函数的执行时间

In [8]:
import time

def function():
    time.sleep(0.8)

function() # 函数调用
print(function())

None


- 装饰器装饰有参函数

In [12]:
import time

def decorator(function):
    
    def inner():
        start_time = time.time()
        function()
        end_time = time.time()
        print(end_time - start_time)

    return inner

@decorator 
def function():
    time.sleep(0.8)

function() # 函数调用

0.8004000186920166


- 装饰器返回值

In [18]:
def celebrator(fun):
    def inner(*args):       #inner必须有参数才能传参,*args，**kwargs可装饰更多函数
        print("我是新增功能")
        ret = fun(*args)
        return ret
    return inner


@celebrator #myprint1  = celebrator(myprint1)   myprint1 = inner; 
def myprint1(a,b):
    print(a,b)
    return a 

@celebrator
def myprint2(a,b):
    print(a,b)
    return b

res1 = myprint1("hhhh",2333) #myprint1() = inner()
res2 = myprint2("give me",5)

print([res1,res2])


我是新增功能
hhhh 2333
我是新增功能
give me 5
['hhhh', 5]


- 双重语法糖,多层嵌套装饰器（闭包函数）

In [19]:
def printequal(func):
    def inner1():
        print("="*15)
        func()
    return inner1
   
def printstar(func):
    def inner2():
        print("*"*15)
        func()
    return inner2

@printequal   #--> myprint printequal(inner2)    return inner1    func:inner2
@printstar  # --> myprint = printstar(myprint)   retrun inner2   func:myprint
def myprint():
    print("哈哈哈") 

myprint()

***************
哈哈哈


## 理解程序过程 执行顺序探究

In [14]:
#思考，输出是什么？
def dec1(func):  
    print("1111")
    def one():
        print("2222")
        func()
        print("3333")
    return one

def dec2(func):   
    print("aaaa")
    def two():
        print("bbbb")
        func()
        print("cccc")
    return two

def dec3(func):    
    print("!!!!")
    def three():
        print("####")
        func()
        print("****")
    return three

@dec1         
@dec2        
@dec3        
def test():
    print("test test")
  
test()

!!!!
aaaa
1111
2222
bbbb
####
test test
****
cccc
3333


- 自学 带参数的装饰糖